In [1]:
import os
import pandas as pd

In [2]:
from census import Census
from us import states

# Note, you must obtain a census key and create a system environment variable called CENSUS_KEY that stores the key value
# You may need to completely restart anaconda/jupyter/VS to get it to recognize the new variable
c = Census(os.environ['CENSUS_KEY'])

census_year = 2023

In [3]:
year = '2023'

In [4]:
def get_acs_data(table, geog):
    results_df = pd.DataFrame()

    df = pd.DataFrame(
        c.acs5.get(table, geo={'for': geog,
                   'in': 'state:{}'.format(states.WA.fips)})
    )

    return df

# Place

In [5]:
place_name_df = pd.DataFrame(c.sf1.get('NAME', geo={'for': 'place:*',
                       'in': 'state:{}'.format(states.WA.fips)})
            )
place_name_df.drop('state', axis=1, inplace=True)
# place_name_df

In [6]:

acs_dict = {
    'B08301_001': 'Total',
    'B08301_003': 'Drove Alone',
    'B08301_004': 'Carpooled',
    'B08301_010': 'Transit',
    'B08301_019': 'Walked',
    'B08301_020': 'Other',
    'B08301_021': 'worked_at_home',
}

df_place = pd.DataFrame()

for table_id, mode in acs_dict.items():

    for data_type, data_id in {'estimate': 'E',
                               'margin_of_error':'M'}.items():
        
        df = get_acs_data(table_id+data_id, 'place:*')
        df['mode'] = mode
        df['data_type'] = data_type
        df.rename(columns={table_id+data_id: 'value'}, inplace=True)
        
        df_place = pd.concat([df_place, df])
    

In [7]:
df_place['geoid'] = df_place['place'].copy()

In [8]:
df_place.rename(columns={'place': 'place_name'}, inplace=True)

In [9]:
# df_place.groupby(['place','state','mode','data_type']).sum()[['']]
df_place = pd.pivot_table(df_place, index=['geoid','state','place_name','mode'], columns=['data_type'], values='value', aggfunc='sum').reset_index()

In [10]:
# df_place.head()

In [11]:
df_place = df_place.merge(place_name_df, left_on='place_name', right_on='place', how='left')

In [12]:
# Only select places needed in region.
# Use existing table as template
# df_template = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\acs_commute_mode_by_workplace_geog.csv')
# df_template = df_template[df_template['year'] != int(year)]
# df_template['geog_name'].unique()

In [13]:
# Note that we are missing many places here;
# The older version must have been trimmed down to only include a select set of places
# df_place = df_place[df_place['NAME'].isin(df_template['geog_name'].unique())]

In [14]:
# df_place.rename(columns={'NAME': 'geog_name'}, inplace=True)

In [15]:
df_place['year'] = year
df_place['place_type'] = 'place'
df_place = df_place[['geoid','place_name','place_type','mode','estimate','year']]

# County

In [16]:
df_county = pd.DataFrame()

for table_id, mode in acs_dict.items():

    for data_type, data_id in {'estimate': 'E',
                               'margin_of_error':'M'}.items():
        
        df = get_acs_data(table_id+data_id, 'county:*')
        df['mode'] = mode
        df['data_type'] = data_type
        df.rename(columns={table_id+data_id: 'value'}, inplace=True)
        
        df_county = pd.concat([df_county, df])
    
county_map = {'033': 'King',
               '035': 'Kitsap',
               '053': 'Pierce',
               '061': 'Snohomish'
              }    
df_county['place_name'] = df_county['county'].map(county_map)
df_county = df_county[~df_county['place_name'].isnull()]  
df_county['geoid'] = df_county['county'].copy()

In [17]:
df_county = pd.pivot_table(df_county, index=['geoid','place_name','mode'], columns=['data_type'], values='value', aggfunc='sum').reset_index()

df_county['year'] = year
df_county['place_type'] = 'county'
df_county = df_county[['geoid','place_name','place_type','mode','estimate','year']]


# Tract

In [23]:
df_tract = pd.DataFrame()

for table_id, mode in acs_dict.items():

    for data_type, data_id in {'estimate': 'E',
                               'margin_of_error':'M'}.items():
        
        df = get_acs_data(table_id+data_id, 'tract:*')
        df['mode'] = mode
        df['data_type'] = data_type
        df.rename(columns={table_id+data_id: 'value'}, inplace=True)
        
        df_tract = pd.concat([df_tract, df])
    
county_map = {'033': 'King',
               '035': 'Kitsap',
               '053': 'Pierce',
               '061': 'Snohomish'
              }    
df_tract['county_name'] = df_tract['county'].map(county_map)
df_tract = df_tract[~df_tract['county_name'].isnull()]   
df_tract['geoid'] = df_tract['state'].astype('str')+df_tract['county']+df_tract['tract']


In [24]:
df_tract = pd.pivot_table(df_tract, index=['geoid','tract','mode'], columns=['data_type'], values='value', aggfunc='sum').reset_index()
df_tract.rename(columns={'tract': 'place_name'}, inplace=True)
df_tract['year'] = year
df_tract['place_type'] = 'tr'
df_tract = df_tract[['place_name','place_type','mode','estimate','year']]


In [26]:
final_df = pd.concat([df_place, df_tract, df_county])

In [220]:
# 

In [29]:
final_df['field1'] = range(1,len(final_df)+1)
final_df.to_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\acs_commute_mode_home_tract.csv',
               index=False)

In [27]:
final_df['place_type'].unique()

array(['place', 'tr', 'county'], dtype=object)

In [28]:
final_df

,geoid,place_name,place_type,mode,estimate,year
0,00100,00100,place,Carpooled,734.0,2023
1,00100,00100,place,Drove Alone,5033.0,2023
2,00100,00100,place,Other,66.0,2023
3,00100,00100,place,Total,6525.0,2023
4,00100,00100,place,Transit,89.0,2023
...,...,...,...,...,...,...
23,061,Snohomish,county,Other,3415.0,2023
24,061,Snohomish,county,Total,418564.0,2023
25,061,Snohomish,county,Transit,15840.0,2023
26,061,Snohomish,county,Walked,7461.0,2023
